<a href="https://colab.research.google.com/github/SC990987/SummerResearch2022/blob/main/QG_SACAMPBELL_Rev1_AddedKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Extract Data


In [1]:
mkdir data/

In [2]:
!wget https://cernbox.cern.ch/index.php/s/ZUHveJKajnZNwTA/download -O data/ZUHveJKajnZNwTA.tar

--2022-08-07 18:12:45--  https://cernbox.cern.ch/index.php/s/ZUHveJKajnZNwTA/download
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.53.35, 128.142.53.28, 137.138.120.151, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.53.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘data/ZUHveJKajnZNwTA.tar’

data/ZUHveJKajnZNwT     [             <=>    ]   4.51G  22.5MB/s    in 3m 56s  

2022-08-07 18:16:49 (19.6 MB/s) - ‘data/ZUHveJKajnZNwTA.tar’ saved [4848198144]



In [3]:
cd /content/data

/content/data


In [4]:
import tarfile
my_tar = tarfile.open('ZUHveJKajnZNwTA.tar')
my_tar.extractall('./data') # specify which folder to extract to
my_tar.close()

In [5]:
cd /content/data

/content/data


# Convert Data Into Images


In [6]:
cd /content/data/data

/content/data/data


In [7]:
import numpy as np
import pyarrow.parquet as pq
import os
import time as time
import PIL.Image as imm
path = "/content/data"
import cv2

In [8]:
import os
os.makedirs("./Images/0")
os.makedirs("./Images/1")

In [9]:
path = "./Images/"

In [10]:
def generate(pf):
    record_batch = pf.iter_batches(batch_size=1024)
    while(True):
        try:
            batch = next(record_batch)
            print(batch.num_rows)
            transform(batch)
        except StopIteration:
            print("Done")
            break

In [11]:
alpha = 0
beta =  0
def saver(im,meta):
    global alpha
    global beta
    for i in range(meta.shape[0]):
        img = im[i,:,:,:]
        img = img.T
        img[:,:,0] = ( img[:,:,0] - img[:,:,0].min() ) / ( img[:,:,0].max()-img[:,:,0].min() )
        img[:,:,1] = ( img[:,:,1] - img[:,:,1].min() ) / ( img[:,:,1].max()-img[:,:,1].min() )
        img[:,:,2] = ( img[:,:,2] - img[:,:,2].min() ) / ( img[:,:,2].max()-img[:,:,2].min() )
        img = img*255
        img = img.astype(np.uint8)
        #lmm = imm.fromarray(img)
        if(meta[i] == 0):
            impath = path+"1/"+str(alpha)+".png"
            alpha = alpha + 1
        if(meta[i] == 1):
            impath = path+"0/"+str(beta)+".png"
            beta = beta + 1
        #lmm.save(impath)
        cv2.imwrite(impath , img)

In [12]:
def transform(batch):
    p = batch.to_pandas()
    im = np.array(np.array(np.array(p.iloc[:,0].tolist()).tolist()).tolist())
    meta = np.array(p.iloc[:,3])
    saver(im,meta)

In [13]:
generate(pq.ParquetFile("QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet"))
generate(pq.ParquetFile("QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet"))
generate(pq.ParquetFile("QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet"))

1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
432
Done
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
436
Done
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
198
Done


# Preprocessing (Pytorch)

In [ ]:
import tensorflow as tf
import numpy as np
np.random.seed(0)
import os, glob
import time
import h5py
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import ConcatDataset, Dataset, DataLoader, sampler, DistributedSampler
from sklearn.metrics import roc_curve, auc
from torch.utils.data import *
import numpy
import time
import h5py
from keras.models import Sequential
from keras.initializers import TruncatedNormal
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
import math 

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

In [ ]:
class ParquetDataset(Dataset):
    def __init__(self, filename, transform=None):
        self.parquet = pq.ParquetFile(filename)
        self.cols = None 
        self.transform = transform
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        # Preprocessing
        start = time.time()
        data['X_jets'] = np.float32(data['X_jets'])
        data['X_jets'] = torch.Tensor(data['X_jets'])
        data['y'] = torch.Tensor(data['y'])
        
        #to preprocess
        data['X_jets'][data['X_jets'] < 1.e-3] = 0. # Zero-Suppression
        data['X_jets'][-1,...] = 25.*data['X_jets'][-1,...] # For HCAL: to match pixel intensity distn of other layers
        data['X_jets'] = data['X_jets']/100. # To standardize
        #data['X'] = np.float32(data['X'][0])
        #print("List lenth",data['X_jets'])
        #data['X_jets'] = numpy.array(data['X_jets'])
        
        if self.transform:
            #data['X_jets'] = numpy.moveaxis(data['X_jets'], 1,3)
            #print("Trnsforn : ",data['X_jets'].shape)
            data['X_jets']= self.transform(data['X_jets'][0]) 
        #data['X_jets'] = numpy.moveaxis(data['X_jets'], -1,1)
        
        #print("Lter : ",data['X_jets'].shape)
        
        end = time.time()
        #print("Time required for preprocessing : ",end - start)
        
        #print("Mean : ", torch.mean(data['X_jets']))
        return dict(data)
    def __len__(self):
        return self.parquet.num_row_groups

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
def get_data_loader(datasets, batch_size, cut, random_sampler=False):

    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(20)
    ])
    #scripted_transforms = torch.jit.script(transforms)
  
    dset = ConcatDataset([ParquetDataset(dataset, transform=transform) for dataset in datasets])
    idxs = np.random.permutation(len(dset))
    if random_sampler: 
        random_sampler = sampler.SubsetRandomSampler(idxs[:cut])
    else: 
        random_sampler = None 
    validation_split = .2
    shuffle_dataset = True
    random_seed= 42

# Creating data indices for training and validation splits:
    dataset_size = 55494
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    print(split)
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    print("Train indices ",train_sampler)
    print("Test indices ", valid_sampler)
    train_loader = torch.utils.data.DataLoader(dset, batch_size=batch_size, 
                                           sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(dset, batch_size=batch_size,
                                                sampler=valid_sampler)
    
    #data_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=10, sampler=random_sampler, pin_memory=True)
    return train_loader, test_loader

In [ ]:
datasets_train = ['QCDToGGQQ_IMGjet_RH1all_jet0_run0_n215556.train.snappy.parquet', 
                  'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n297980.train.snappy.parquet', 
                  'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n280364.train.snappy.parquet']
datasets_test = ['QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet',
                 'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet',
                 'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet']

In [ ]:
cd /content/data/data

/content/data/data


In [ ]:
train_loader, test_loader= get_data_loader(datasets_train, 32, cut = None, random_sampler = True)
#test_loader = get_data_loader(datasets_test, 32, cut = None, random_sampler = True)

11098
Train indices  <torch.utils.data.sampler.SubsetRandomSampler object at 0x7fa306872e50>
Test indices  <torch.utils.data.sampler.SubsetRandomSampler object at 0x7fa306872590>


In [ ]:
for i, batch in enumerate(train_loader):
    X_jets,	pt,	m0,	y = batch["X_jets"], batch["pt"], batch["m0"],batch["y"]
    #print(torch.max(X_jets))
    if (i%50 ==0):
        print(torch.max(X_jets))

tensor(61.3017)
tensor(123.9118)
tensor(85.9522)
tensor(70.9921)
tensor(63.5722)
tensor(71.1913)
tensor(76.9099)
tensor(67.4820)
tensor(103.7840)
tensor(282.7818)
tensor(67.6818)
tensor(96.1544)
tensor(94.8591)
tensor(76.1987)
tensor(53.1465)
tensor(136.4543)
tensor(84.2105)
tensor(58.4805)
tensor(61.8126)
tensor(71.4254)
tensor(72.6430)
tensor(66.9177)
tensor(71.6387)
tensor(35.5399)
tensor(115.7959)
tensor(40.2482)
tensor(69.3265)
tensor(59.3713)


In [ ]:
def my_collate(batch):
    data = torch.cat([item[0] for item in batch],dim = 0)
    target = torch.cat([item[1] for item in batch],dim = 0)

    return [data, target]

In [ ]:
def train_epoch(model, optimizer, data_loader, loss_history):
    total_samples = len(data_loader)*32 #batch size =32
    correct_samples = 0
    model.train()
    
    for i, batch in enumerate(data_loader):
        optimizer.zero_grad()
        data = batch["X_jets"]
        data = data.cuda()
        output = model(data)
        target = batch["y"]
        target = target.cuda()
        target = torch.squeeze(target, dim=1)
        loss = nn.BCEWithLogitsLoss()(torch.squeeze(output, dim=1),target)
        pred = torch.squeeze(nn.Sigmoid()(output))
        pred[pred>=0.5] = 1
        pred[pred<0.5] = 0
        optimizer.step()
        
        correct_samples += pred.eq(target).sum()

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()) + '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
                  '{:5}'.format(total_samples) + ' (' + '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)')
            loss_history.append(loss.item())

In [ ]:
def evaluate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = len(data_loader)*32 #len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            
            data = batch["X_jets"]
            data = data.cuda()
            output = model(data)
            target = batch["y"]
            target = target.cuda()
            target = torch.squeeze(target, dim=1)
            loss = nn.BCEWithLogitsLoss()(torch.squeeze(output, dim=1),target)
            pred = torch.squeeze(nn.Sigmoid()(output))
            pred[pred>=0.5] = 1
            pred[pred<0.5] = 0
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / (total_samples/32)
    loss_history.append(avg_loss) 
    print('\nAverage val loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

# Model Selection, Training, and Evaluation (Pytorch)

In [ ]:
import torchvision.models as models
import torchvision
from torchvision.models import ResNet18_Weights
torchvision.models.ResNet18_Weights(ResNet18_Weights.DEFAULT)
#resnet18 = models.resnet18(weights='IMAGENET1K_V1')
resnet18 = models.resnet18(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
N_EPOCHS = 8
import time
start_time = time.time()
from torch import nn
from sklearn.metrics import roc_curve, auc
model = resnet18
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
model.fc = nn.Sequential(nn.AdaptiveAvgPool1d(512),
                         nn.Dropout(0.3),
                         nn.Linear(512,256),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(256,1),
                         nn.Sigmoid())
model.cuda()

cnt = 0
train_loss_history, test_loss_history = [], []
for epoch in range(1, N_EPOCHS + 1):
    
    r = 0.0005/(10**(cnt))
    print("LR:",r)
    optimizer = torch.optim.Adam(model.parameters(), lr=r)
    print('Epoch:', epoch)
    train_epoch(model, optimizer, train_loader, train_loss_history)
    print("Training done!")
    evaluate(model, test_loader, test_loss_history)
    pred_list, target_list = evaluate(model, test_loader, test_loss_history)
    fpr, tpr = roc_curve(target_list, pred_list)
    roc_auc = auc(fpr, tpr)
    print('Training ROC AUC:', roc_auc)
    if epoch%4==0:
      cnt=cnt+1
    

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

LR: 0.0005
Epoch: 1
[    0/44416 (  0%)]  Loss: 0.7515  Accuracy:   15/44416 (0.03%)
[ 3200/44416 (  7%)]  Loss: 0.6975  Accuracy: 1579/44416 (3.56%)
[ 6400/44416 ( 14%)]  Loss: 0.6463  Accuracy: 3208/44416 (7.22%)
[ 9600/44416 ( 22%)]  Loss: 0.7541  Accuracy: 4803/44416 (10.81%)
[12800/44416 ( 29%)]  Loss: 0.7664  Accuracy: 6396/44416 (14.40%)
[16000/44416 ( 36%)]  Loss: 0.7962  Accuracy: 8001/44416 (18.01%)
[19200/44416 ( 43%)]  Loss: 0.7370  Accuracy: 9609/44416 (21.63%)
[22400/44416 ( 50%)]  Loss: 0.6639  Accuracy:11221/44416 (25.26%)
[25600/44416 ( 58%)]  Loss: 0.7170  Accuracy:12805/44416 (28.83%)
[28800/44416 ( 65%)]  Loss: 0.6403  Accuracy:14400/44416 (32.42%)
[32000/44416 ( 72%)]  Loss: 0.7216  Accuracy:15984/44416 (35.99%)
[35200/44416 ( 79%)]  Loss: 0.7236  Accuracy:17598/44416 (39.62%)
[38400/44416 ( 86%)]  Loss: 0.7387  Accuracy:19162/44416 (43.14%)
[41600/44416 ( 94%)]  Loss: 0.7532  Accuracy:20815/44416 (46.86%)
Training done!

Average val loss: 0.7293  Accuracy: 5553/11

TypeError: ignored

In [ ]:
def test(model, data_loader, loss_history):
    model.eval()
    pred_list = []
    target_list = []
    total_samples = len(data_loader)*32
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            
            data = batch["X_jets"]
            data = data.cuda()
            output = model(data)
            target = batch["y"]
            target = target.cuda()          
            target = torch.squeeze(target, dim=1)
            loss = nn.BCEWithLogitsLoss()(torch.squeeze(output, dim=1),target)
            pred = torch.squeeze(nn.Sigmoid()(output))
            pred_list.extend(pred.tolist())
            target_list.extend(target.tolist())
            pred[pred>=0.5] = 1
            pred[pred<0.5] = 0
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / (total_samples/32)
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')
    return pred_list,target_list

In [ ]:
test_loss_history = []
pred_list, target_list = test(model, test_loader, test_loss_history)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(target_list, pred_list)
roc_auc = auc(fpr, tpr)
print('Test ROC AUC:', roc_auc)

In [ ]:
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], 'k--')
#plt.legend(loc=2, prop={'size': 15})
plt.plot(fpr, tpr, label='Model 1 (ROC-AUC = {:.3f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve: Resnet 18 (modified FC Layer)')
plt.legend(loc='best')
plt.show()
plt.savefig('resnet_18_ROC_modified.png')

# Preprocessing (Keras)

In [23]:
import numpy as np
import PIL.Image as im
import tensorflow as tf
import tensorflow.keras.layers as tfl
import matplotlib.pyplot as plt
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.utils.vis_utils import plot_model

In [24]:
train_datagen= ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)
train_generator = train_datagen.flow_from_directory(
        '/content/data/data/Images',
        target_size= (125,125),
        color_mode='rgb',
        batch_size=32,
        class_mode='binary',
        subset='training',
        shuffle = True
)

validation_generator = train_datagen.flow_from_directory(
        '/content/data/data/Images',
        target_size= (125,125),
        color_mode='rgb',
        batch_size=32,
        class_mode='binary',
        subset='validation',
        shuffle = True

)

Found 111446 images belonging to 2 classes.
Found 27860 images belonging to 2 classes.


# Model Selection, Training, and Evaluation (Keras)

In [40]:
base_model = tf.keras.applications.resnet50.ResNet50(input_shape= (125,125,3), 
                                                  weights  = "imagenet",
                                                  include_top = False
                                                 )

94781440/94765736 [==============================] - 2s 0us/step


In [41]:
base_model.trainable = True

In [42]:
inputs = tf.keras.Input(shape=(125,125,3))
x = tf.keras.layers.RandomFlip()(inputs)
x = tf.keras.layers.RandomRotation(0.2)(x)
x = base_model(x, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(256 , activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

In [43]:
pip install -q -U tensorflow_addons

In [44]:
from tensorflow_addons.optimizers import CyclicalLearningRate
cyclical_learning_rate = CyclicalLearningRate(
 initial_learning_rate=1e-6,
 maximal_learning_rate=1e-3,
 step_size=1000*6,
 scale_fn=lambda x: 1 / (2.0 ** (x - 1)),
 scale_mode='cycle')

In [45]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=cyclical_learning_rate ,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name = "auc")]
)

In [46]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./cp_{epoch}_.ckpt",
    monitor='val_auc',
    verbose=1,
    save_weights_only=True,
    mode='auto',
    save_freq = 1000
)

In [ ]:
history = model.fit(
        train_generator,
        validation_data = validation_generator,
        validation_steps = 500,
        epochs=200,
        steps_per_epoch = 1000,
        verbose = 2,
        callbacks = [checkpoint]
)

Epoch 1/200

Epoch 1: saving model to ./cp_1_.ckpt
1000/1000 - 172s - loss: 0.6507 - auc: 0.6995 - val_loss: 0.5928 - val_auc: 0.7535 - 172s/epoch - 172ms/step
Epoch 2/200

Epoch 2: saving model to ./cp_2_.ckpt
1000/1000 - 157s - loss: 0.6033 - auc: 0.7442 - val_loss: 0.5905 - val_auc: 0.7558 - 157s/epoch - 157ms/step
Epoch 3/200

Epoch 3: saving model to ./cp_3_.ckpt
1000/1000 - 159s - loss: 0.5982 - auc: 0.7483 - val_loss: 0.5951 - val_auc: 0.7526 - 159s/epoch - 159ms/step
Epoch 4/200

Epoch 4: saving model to ./cp_4_.ckpt
1000/1000 - 159s - loss: 0.5957 - auc: 0.7507 - val_loss: 0.5910 - val_auc: 0.7555 - 159s/epoch - 159ms/step
Epoch 5/200

Epoch 5: saving model to ./cp_5_.ckpt
1000/1000 - 159s - loss: 0.5951 - auc: 0.7519 - val_loss: 0.5972 - val_auc: 0.7604 - 159s/epoch - 159ms/step
Epoch 6/200

Epoch 6: saving model to ./cp_6_.ckpt
1000/1000 - 159s - loss: 0.5930 - auc: 0.7540 - val_loss: 0.5857 - val_auc: 0.7621 - 159s/epoch - 159ms/step
Epoch 7/200

Epoch 7: saving model to ./

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(history.history).to_csv('history.csv',index=False)

In [ ]:
history = pd.read_csv("./History.csv")

In [ ]:
plt.plot(np.array(range(0,200)), np.array(history["loss"]) , label = "Training")
plt.plot(np.array(range(0,200)), np.array(history["val_loss"]) , label = "Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Validation and Training loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.array(range(0,200)), np.array(history["auc"]) , label = "Training")
plt.plot(np.array(range(0,200)), np.array(history["val_auc"]) , label = "Validation")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.title("Validation and Training AUC")
plt.legend()
plt.show()

In [ ]:
step = np.arange(0, 200 * 1000)
lr = cyclical_learning_rate(step)
plt.plot(step, lr)
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.title("Learning Rate")
plt.show()